In [ ]:
import os
os.environ["GOOGLE_API_KEY"] = ""

In [ ]:
pip install pydantic==1.10.7

In [ ]:
! pip install -q ragstack-ai datasets

In [ ]:
import cassio

In [ ]:
import os
from getpass import getpass

# Enter your settings for Astra DB and OpenAI:
os.environ["ASTRA_DB_API_ENDPOINT"] =""
os.environ["ASTRA_DB_APPLICATION_TOKEN"] =""
os.environ["GEMINI_API_KEY"] = getpass("Enter your Gemini API Key: ")

Enter your Gemini API Key: ··········


In [ ]:
from langchain_astradb import AstraDBVectorStore
from langchain_google_genai import GoogleGenerativeAIEmbeddings
import os

# Configure your embedding model and vector store
embedding = GoogleGenerativeAIEmbeddings(model="models/embedding-001")
vstore = AstraDBVectorStore(
    collection_name="geminii",
    embedding=embedding,
    token=os.getenv("ASTRA_DB_APPLICATION_TOKEN"),
    api_endpoint=os.getenv("ASTRA_DB_API_ENDPOINT"),
)
print("Astra vector store configured")

Astra vector store configured


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
with open('/content/drive/Shareddrives/DATA 298A/Code/Output- Preprocessed Data/Combined_Content_From_PDF.txt', 'r', encoding='utf-8') as file:
    content = file.read()

In [ ]:
pages = content.split("\f")

In [ ]:
# Initialize an empty string to store concatenated text
raw_text = ''

# Iterate over each "page" and concatenate the content
for i, page_content in enumerate(pages):
    if page_content.strip():  # Check if the page has any content
        raw_text += page_content + "\n"

In [ ]:
from langchain.text_splitter import CharacterTextSplitter
# We need to split the text using Character Text Split such that it should not increse token size
text_splitter = CharacterTextSplitter(
    separator = "\n",
    chunk_size = 512,
    chunk_overlap  = 150,
    length_function = len,
)
texts = text_splitter.split_text(raw_text)

In [ ]:
from langchain.indexes.vectorstore import VectorStoreIndexWrapper
vstore.add_texts(texts)

print("Number of headline inserted in AstraDB",  len(texts))

astra_vector_index = VectorStoreIndexWrapper(vectorstore=vstore)

Number of headline inserted in AstraDB 282


In [ ]:
# Checks your collection to verify the documents are embedded.
print(vstore.astra_db.collection("geminii").find())

{'data': {'documents': [{'_id': '647a07bcfbae4f60abcffc9b937af837', 'content': 'response to anticipated public scrutiny, Facebook is likely to reinstitute such\npolicies if such scrutiny passes.  44\nCase 1:20-cv-03590-JEB Document 51 Filed 01/13/21 Page 45 of 53  150. Because access\nto the relevant APIs is valuable to app developers, Facebook policy  conditions\nchanged the incentives of app developers, and deterred them from developing\ncompeting functionalities or supporting competing social networks.  151. Moreover,', 'metadata': {}}, {'_id': '499cee949f074bfba1a6752e2650ec74', 'content': 'acquisition: “new products  22\nCase 1:20-cv-03590-JEB Document 51 Filed 01/13/21 Page 23 of 53  won’t get much\ntraction since we’ll already have their mechanics deployed at scale.”  Facebook\ncontinues to look for other competitive threats, and will seek to acquire them unless\n insight into the most  popular apps.  We should use that to also help us make\nstrategic acquisitions.” Facebook als

In [ ]:
from langchain.prompts import ChatPromptTemplate
from langchain_google_genai import GoogleGenerativeAI
from langchain.schema.output_parser import StrOutputParser
from langchain.schema.runnable import RunnablePassthrough

retriever = vstore.as_retriever(search_kwargs={"k": 3})

prompt_template = """
Answer the question based only on the supplied context. If you don't know the answer, say you don't know the answer.
Context: {context}
Question: {question}
Your answer:
"""
prompt = ChatPromptTemplate.from_template(prompt_template)
model = GoogleGenerativeAI(model="gemini-pro")

chain = (
    {"context": retriever, "question": RunnablePassthrough()}
    | prompt
    | model
    | StrOutputParser()
)

In [ ]:
chain.invoke("Give a summary about complaint case filed against Facebook")

"The Federal Trade Commission (FTC) filed a complaint against Facebook, Inc., alleging that the company has engaged in anticompetitive conduct and unfair methods of competition in violation of Section 5(a) of the FTC Act. The FTC seeks permanent injunctive relief to remedy Facebook's alleged violations of the law."

In [ ]:
chain.invoke("what is the weather looking like today?")

"I don't know the answer to this question."

In [ ]:
chain.invoke("Who filed the case against Facebook and what was the case?")

'I do not know the answer to this question.'